In [ ]:
from pyspark.sql.types import IntegerType
# importing pyspark
import pyspark
  
# importing SparkSession
from pyspark.sql import SparkSession
  
# importing all from pyspark.sql.function
from pyspark.sql.functions import *

In [ ]:
###### Mount Point 1 through Oauth security.
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group6-data"
clientSecret = 
clientid = 
mount_point = "/mnt/gavanvanover/capstone-group6-populating"
#20200906-20201006/Detroit911-20200906-20201006.csv
#@28077959-1e7b-474a-995f-492292e0b7f8
#Copy to clipboard

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(
source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
mount_point = mount_point,
extra_configs = configs)

/mnt/gavanvanover/capstone-group6-populating has been unmounted.
Out[2]: True

## Create dataframes from the MACRO and CPI data sets

In [ ]:
display(dbutils.fs.ls("/mnt/gavanvanover/capstone-group6-populating"))

path,name,size,modificationTime
dbfs:/mnt/gavanvanover/capstone-group6-populating/Salestable_final.csv,Salestable_final.csv,553423,1660244005000
dbfs:/mnt/gavanvanover/capstone-group6-populating/USACPIALLMINMEI.csv,USACPIALLMINMEI.csv,20709,1659496423000
dbfs:/mnt/gavanvanover/capstone-group6-populating/US_MACRO.csv,US_MACRO.csv,627940,1659496423000
dbfs:/mnt/gavanvanover/capstone-group6-populating/retail_census_data_NAICS_NAPCS_2017.csv,retail_census_data_NAICS_NAPCS_2017.csv,675608,1659496178000


## CPI Data Set

In [ ]:
cpi_df = spark.read.csv('/mnt/gavanvanover/capstone-group6-populating/USACPIALLMINMEI.csv', header = 'True')

# Create year and month columns from the date
cpi_df2 = cpi_df.withColumn('Year2', split(cpi_df['DATE'], '-').getItem(0)) \
       .withColumn('Month2', split(cpi_df['DATE'], '-').getItem(1))
# Rename column to CPI
cpi_df2 = cpi_df2.withColumnRenamed('USACPIALLMINMEI', 'CPI')
# Drop the date column
cpi_df3 = cpi_df2.drop(col('DATE'))
# Remove the leading zeroes from the month column
cpi_df3 = cpi_df3.withColumn('Month2', pyspark.sql.functions.regexp_replace('Month2', r'^[0]*', ''))


cpi_df3.dtypes

Out[4]: [('CPI', 'string'), ('Year2', 'string'), ('Month2', 'string')]

## Macroeconomics Dataset

In [ ]:
macro_df = spark.read.csv('/mnt/gavanvanover/capstone-group6-populating/US_MACRO.csv', header = 'True')

# Keep only the columns we want
macro_df_cleaned = macro_df['observation_date','RPI', 'RETAIL', 'USTRADE', 'USWTRADE']
# Rename observation_date to date
macro_df_cleaned = macro_df_cleaned.withColumnRenamed('observation_date', 'DATE')
# Create columns for Year and Month from the Date Column
macro_df_cleaned = macro_df_cleaned.withColumn('Year', split(macro_df_cleaned['DATE'], '/').getItem(2)) \
       .withColumn('Month', split(macro_df_cleaned['DATE'], '/').getItem(0))
# Drop the Date Column
macro_df_cleaned = macro_df_cleaned.drop(col('DATE'))

macro_df_cleaned.dtypes

Out[5]: [('RPI', 'string'),
 ('RETAIL', 'string'),
 ('USTRADE', 'string'),
 ('USWTRADE', 'string'),
 ('Year', 'string'),
 ('Month', 'string')]

## Merge the dataframes

In [ ]:
# Join the macro and CPI Data on the Month and Year
merged_df = macro_df_cleaned.join(cpi_df3, (macro_df_cleaned.Year == cpi_df3.Year2) & (macro_df_cleaned.Month == cpi_df3.Month2), 'full')
# Drop the copied columns
merged_df = merged_df.drop('Year2','Month2')
# Convert Month and Year to Integers
merged_df_cleaned = merged_df.withColumn('Month',merged_df.Month.cast(IntegerType())) \
    .withColumn('Year',merged_df.Year.cast(IntegerType()))
# Sort the data in time order
merged_and_ordered_df = merged_df_cleaned.orderBy(col('Year').asc(), col('Month').asc())
merged_and_ordered_df.show()

+--------+-----------+-------+--------+----+-----+----------------+
 RPI| RETAIL|USTRADE|USWTRADE|Year|Month| CPI|
+--------+-----------+-------+--------+----+-----+----------------+
 null| null|15881.5| 5835.7|null| null| null|
2442.158|18235.77392| 5350.3| 2568.7|1959| 1| null|
2451.778|18369.56308| 5381.3| 2575.4|1959| 2| null|
2467.594|18523.05762| 5431.6| 2584.4|1959| 3| null|
2483.671| 18534.466| 5395.4| 2596.9|1959| 4| null|
2498.026|18679.66354| 5456.3| 2607.4|1959| 5| null|
2505.788|18849.75209| 5467.8| 2614.8|1959| 6| null|
2504.312|18843.52934| 5471.4| 2617|1959| 7| null|
2490.236|18963.83587| 5489| 2657.7|1959| 8| null|
2492.033|18715.96293| 5484.4| 2640.4|1959| 9| null|
2495.225|18852.86346| 5494.3| 2644.5|1959| 10| null|
2514.248|18356.08045| 5503.7| 2650|1959| 11| null|
2540.011|18271.03618| 5527.9| 2661.7|1959| 12| null|
2548.137|18763.67069| 5554.1| 2657.9|1960| 1| 12.361982473831|
 2551.05|18833.15808| 5578| 2662.6|1960| 2|12.4041735402946|
2555.924|18812.41558| 5573| 2666.8|1960| 3|12.4041735402946|
2564.677|19306.08721| 5671.2| 2677.3|1960| 4|12.4463646067582|
2570.841|19017.76638| 5596.8| 2671.7|1960| 5|12.4463646067582|
2572.869|18991.83825| 5594.8| 2668|1960| 6|12.4885556732217|
2577.579| 18801.0072| 5597.3| 2667|1960| 7|12.4885556732217|
+--------+-----------+-------+--------+----+-----+----------------+
only showing top 20 rows

In [ ]:
# Create a dataframe to store the Year and Month
time_df = merged_and_ordered_df['Year','Month']
#time_df.show()
# Create a dataframe to store the macroeconomic and CPI data
macro_and_cpi_df = merged_and_ordered_df['CPI','RPI','USTRADE','USWTRADE']
macro_and_cpi_df.show()

+----------------+--------+-------+--------+
 CPI| RPI|USTRADE|USWTRADE|
+----------------+--------+-------+--------+
 null| null|15881.5| 5835.7|
 null|2442.158| 5350.3| 2568.7|
 null|2451.778| 5381.3| 2575.4|
 null|2467.594| 5431.6| 2584.4|
 null|2483.671| 5395.4| 2596.9|
 null|2498.026| 5456.3| 2607.4|
 null|2505.788| 5467.8| 2614.8|
 null|2504.312| 5471.4| 2617|
 null|2490.236| 5489| 2657.7|
 null|2492.033| 5484.4| 2640.4|
 null|2495.225| 5494.3| 2644.5|
 null|2514.248| 5503.7| 2650|
 null|2540.011| 5527.9| 2661.7|
 12.361982473831|2548.137| 5554.1| 2657.9|
12.4041735402946| 2551.05| 5578| 2662.6|
12.4041735402946|2555.924| 5573| 2666.8|
12.4463646067582|2564.677| 5671.2| 2677.3|
12.4463646067582|2570.841| 5596.8| 2671.7|
12.4885556732217|2572.869| 5594.8| 2668|
12.4885556732217|2577.579| 5597.3| 2667|
+----------------+--------+-------+--------+
only showing top 20 rows

## Database Information

In [ ]:
database = "capstone-group6-database"
table1 = "dbo.MacroTable"
table2 = "dbo.DateTable"
table3 = "dbo.NAICS_NAPCS"
table4 = "dbo.NAICSTable"
table5 = "dbo.NAPCSTable"
table6 = "dbo.SalesTable"
user = 
password  = 
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

## Populate the date table

In [ ]:
time_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option('truncate', 'true') \
    .mode("append") \
    .option("dbtable", table2) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()
    #.option('truncate', 'true') \

In [ ]:
dateid_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table2) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
dateid_df.show()

+------+----+-----+
DateID|Year|Month|
+------+----+-----+
 1|null| null|
 2|1959| 1|
 3|1959| 2|
 4|1959| 3|
 5|1959| 4|
 6|1959| 5|
 7|1959| 6|
 8|1959| 7|
 9|1959| 8|
 10|1959| 9|
 11|1959| 10|
 12|1959| 11|
 13|1959| 12|
 14|1960| 1|
 15|1960| 2|
 16|1960| 3|
 17|1960| 4|
 18|1960| 5|
 19|1960| 6|
 20|1960| 7|
+------+----+-----+
only showing top 20 rows

In [ ]:
# Join Date table and Macro/CPI Data so it will have the DateID foreign key (Join on month and year)
macro_and_dateid_df = merged_and_ordered_df.join(dateid_df, ['Year','Month'], 'full')
# Drop the row with an empty dateid
macro_and_dateid_df = macro_and_dateid_df.na.drop(subset = 'DateID')
# Drop the month and year columns
macro_table_df = macro_and_dateid_df['DateID', 'CPI', 'RPI', 'USTRADE', 'USWTRADE']
macro_table_df = macro_table_df.na.drop(subset = 'RPI')
macro_table_df.show()

+------+----------------+--------+-------+--------+
DateID| CPI| RPI|USTRADE|USWTRADE|
+------+----------------+--------+-------+--------+
 2| null|2442.158| 5350.3| 2568.7|
 3| null|2451.778| 5381.3| 2575.4|
 4| null|2467.594| 5431.6| 2584.4|
 5| null|2483.671| 5395.4| 2596.9|
 6| null|2498.026| 5456.3| 2607.4|
 7| null|2505.788| 5467.8| 2614.8|
 8| null|2504.312| 5471.4| 2617|
 9| null|2490.236| 5489| 2657.7|
 10| null|2492.033| 5484.4| 2640.4|
 11| null|2495.225| 5494.3| 2644.5|
 12| null|2514.248| 5503.7| 2650|
 13| null|2540.011| 5527.9| 2661.7|
 14| 12.361982473831|2548.137| 5554.1| 2657.9|
 15|12.4041735402946| 2551.05| 5578| 2662.6|
 16|12.4041735402946|2555.924| 5573| 2666.8|
 17|12.4463646067582|2564.677| 5671.2| 2677.3|
 18|12.4463646067582|2570.841| 5596.8| 2671.7|
 19|12.4885556732217|2572.869| 5594.8| 2668|
 20|12.4885556732217|2577.579| 5597.3| 2667|
 21|12.4885556732217|2573.388| 5592.3| 2691.3|
+------+----------------+--------+-------+--------+
only showing top 20 rows

## Populate the macro table

In [ ]:
macro_table_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option('truncate', 'true') \
    .mode("overwrite") \
    .option("dbtable", table1) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

## Read in the data from the consumed folder and turn them into dataframes

Create a mount point

In [ ]:
storageAccount2 = "gen10datafund2205"
storageContainer2 = "capstone-group6-consumed"
mount_point2 = "/mnt/gavanvanover/consumed_data_for_use"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point2)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer2}@{storageAccount2}.dfs.core.windows.net/", 
mount_point = mount_point2, 
extra_configs = configs2)

/mnt/gavanvanover/consumed_data_for_use has been unmounted.
Out[13]: True

In [ ]:
display(dbutils.fs.ls("/mnt/gavanvanover/consumed_data_for_use"))

path,name,size,modificationTime
dbfs:/mnt/gavanvanover/consumed_data_for_use/NAICS_NAPCS_2017.csv/,NAICS_NAPCS_2017.csv/,0,1660141174000
dbfs:/mnt/gavanvanover/consumed_data_for_use/NAICS_meanings.csv/,NAICS_meanings.csv/,0,1660141179000
dbfs:/mnt/gavanvanover/consumed_data_for_use/NAPCS_meanings.csv/,NAPCS_meanings.csv/,0,1660141190000


Read in the csvs

In [ ]:
census_df = spark.read.csv('/mnt/gavanvanover/consumed_data/NAICS_NAPCS_2017.csv', header = 'True')
naics_df = spark.read.csv('/mnt/gavanvanover/consumed_data/NAICS_meanings.csv', header = 'True')
napcs_df = spark.read.csv('/mnt/gavanvanover/consumed_data/NAPCS_meanings.csv', header = 'True')
naics_df.show()

+---------------------------+---------------------------------------+
2017 NAICS code (NAICS2017)|Meaning of NAICS code (NAICS2017_LABEL)|
+---------------------------+---------------------------------------+
 444130| Hardware stores|
 454110| Electronic shoppi...|
 445210| Meat markets|
 448190| Other clothing st...|
 445220| Fish and seafood ...|
 448310| Jewelry stores|
 448120| Women's clothing ...|
 453910| Pet and pet suppl...|
 441310| Automotive parts ...|
 443141| Household applian...|
 443142| Electronics stores|
 444210| Outdoor power equ...|
 445120| Convenience stores|
 444220| Nursery, garden c...|
 453210| Office supplies a...|
 442299| All other home fu...|
 451130| Sewing, needlewor...|
 445230| Fruit and vegetab...|
 442210| Floor covering st...|
 448130| Children's and in...|
+---------------------------+---------------------------------------+
only showing top 20 rows

Write to the database

In [ ]:
napcs_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option('truncate', 'true') \
    .mode("append") \
    .option("dbtable", table5) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()
    #.option('truncate', 'true') \


In [ ]:
naics_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option('truncate', 'true') \
    .mode("append") \
    .option("dbtable", table4) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()
    #.option('truncate', 'true') \

In [ ]:
napcs_id_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table5) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
napcs_id_df.show()

+-------+--------------------------------------+--------------------------------------------------+
NAPCSID|2017 NAPCS collection code (NAPCS2017)|Meaning of NAPCS collection code (NAPCS2017_LABEL)|
+-------+--------------------------------------+--------------------------------------------------+
 1| 4000750000| Wholesale sales o...|
 2| 3001650000| Construction serv...|
 3| 4000550000| Wholesale sales o...|
 4| 2001425000| Manufacturing of ...|
 5| 2054075000| Lapidary work, ex...|
 6| 2045275000| Manufacturing of ...|
 7| 2001450000| Manufacturing of ...|
 8| 4001150000| Wholesale sales o...|
 9| 4000725000| Wholesale sales o...|
 10| 2051575000| Embroidery servic...|
 11| 2034910000| Sales of plants, ...|
 12| 2005600000| Manufacturing of ...|
 13| 2053200000| Screen printing o...|
 14| 2052350000| General book prin...|
 15| 4000525000| Wholesale sales o...|
 16| 4000775000| Wholesale sales o...|
 17| 2017950000| Manufacturing of ...|
 18| 4000025000| Wholesale sales o...|
 19| 4000700000| Wholesale sales o...|
 20| 4000675000| Wholesale sales o...|
+-------+--------------------------------------+--------------------------------------------------+
only showing top 20 rows

In [ ]:
naics_id_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table4) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
naics_id_df.show()

+-------+---------------------------+---------------------------------------+
NAICSID|2017 NAICS code (NAICS2017)|Meaning of NAICS code (NAICS2017_LABEL)|
+-------+---------------------------+---------------------------------------+
 1| 444130| Hardware stores|
 2| 454110| Electronic shoppi...|
 3| 445210| Meat markets|
 4| 448190| Other clothing st...|
 5| 445220| Fish and seafood ...|
 6| 448310| Jewelry stores|
 7| 448120| Women's clothing ...|
 8| 453910| Pet and pet suppl...|
 9| 441310| Automotive parts ...|
 10| 443141| Household applian...|
 11| 443142| Electronics stores|
 12| 444210| Outdoor power equ...|
 13| 445120| Convenience stores|
 14| 444220| Nursery, garden c...|
 15| 453210| Office supplies a...|
 16| 442299| All other home fu...|
 17| 451130| Sewing, needlewor...|
 18| 445230| Fruit and vegetab...|
 19| 442210| Floor covering st...|
 20| 448130| Children's and in...|
+-------+---------------------------+---------------------------------------+
only showing top 20 rows

## Add the NAPCS ID and NAICS ID to the census table

In [ ]:
# Join Census Data with the NAICS table to get the NAICS ID
census_and_naics_id_df = census_df.join(naics_id_df, ['2017 NAICS code (NAICS2017)'], 'full')
# Drop the NAICS Code Column
census_and_naics_id_df = census_and_naics_id_df['NAICSID', '2017 NAPCS collection code (NAPCS2017)', 'Number of establishments (ESTAB)','Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)']
census_and_naics_id_df.show()

+-------+--------------------------------------+--------------------------------+----------------------------------------------------------------------------------+
NAICSID|2017 NAPCS collection code (NAPCS2017)|Number of establishments (ESTAB)|Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)|
+-------+--------------------------------------+--------------------------------+----------------------------------------------------------------------------------+
 37| 4001325000| 14157| 22814112|
 37| 4001350000| 9883| 16180114|
 37| 4001950000| null| null|
 37| 4004175000| 59| 121739|
 37| 4004200000| null| null|
 37| 4004225000| 3| 98|
 37| 4004250000| 20| 5479|
 37| 4005500000| 1531| 2303392|
 47| 2054080000| null| null|
 47| 4001325000| 6845| 4954964|
 47| 4001350000| 4103| 1240090|
 47| 4001375000| 3| 263|
 47| 4001950000| 6| 1172|
 47| 4001975000| null| null|
 47| 4004150000| null| null|
 47| 4004175000| 10| 5631|
 47| 4004250000| 17| 209|
 47| 4005500000| 128| 63819|
 26| 4001325000| 17| 611|
 26| 4001350000| 17| 2315|
+-------+--------------------------------------+--------------------------------+----------------------------------------------------------------------------------+
only showing top 20 rows

In [ ]:
# Join the previous dataframe with the NAPCS table to get the NAPCS ID
complete_census_df = census_and_naics_id_df.join(napcs_id_df, ['2017 NAPCS collection code (NAPCS2017)'], 'full')
# Drop the NAPCS Code Column
complete_census_df = complete_census_df['NAICSID', 'NAPCSID', 'Number of establishments (ESTAB)','Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)']
#complete_census_df2 = complete_census_df.drop('2017 NAPCS collection code (NAPCS2017)', 'Meaning of NAPCS collection code (NAPCS2017_LABEL)')
#complete_census_df = complete_census_df.na.drop(subset = ['Number of establishments (ESTAB)', 'Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)'])
#complete_census_df = complete_census_df.na.drop("any")
complete_census_df.show()

+-------+-------+--------------------------------+----------------------------------------------------------------------------------+
NAICSID|NAPCSID|Number of establishments (ESTAB)|Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)|
+-------+-------+--------------------------------+----------------------------------------------------------------------------------+
 13| 51| null| null|
 3| 61| 19| 1485|
 29| 31| 22055| 3710481|
 13| 31| 305| 6087|
 18| 31| 28| 3836|
 49| 31| 110| 6696|
 29| 57| 18810| 4443845|
 13| 57| 131| 2965|
 49| 57| 186| 7339|
 29| 4| 13300| 531701|
 13| 4| 90| 586|
 49| 4| 184| 7290|
 29| 7| 15684| 1780528|
 13| 7| 306| 8699|
 49| 7| 113| 1865|
 25| 7| 6| 325|
 29| 38| 14356| 740953|
 13| 38| 195| 1763|
 49| 38| 90| 2179|
 25| 38| 6| 174|
+-------+-------+--------------------------------+----------------------------------------------------------------------------------+
only showing top 20 rows

In [ ]:
complete_census_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table3) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option('truncate', 'true') \
    .save()

## Read the Sales Table by month csv into a dataframe then to the database

In [ ]:
sales_df = spark.read.csv('/mnt/gavanvanover/capstone-group6-populating/Salestable_final.csv', header = 'True')
sales_df.show()

+---+----+-----+---------+--------------------+----------------+--------------+
_c0|Year|Month|NAICScode| NAICSname|Unadjusted Sales|Adjusted Sales|
+---+----+-----+---------+--------------------+----------------+--------------+
 0|2021| 1| 441|Motor vehicle and...| 100629.0| 115032.0|
 1|2021| 1| 4413|Automotive parts,...| 7686.0| 8446.0|
 2|2021| 1| 442|Furniture and hom...| 10432.0| 11855.0|
 3|2021| 1| 443|Electronics and a...| 6843.0| 7382.0|
 4|2021| 1| 444|Building mat. and...| 30890.0| 38682.0|
 5|2021| 1| 4441|Building mat. and...| 26897.0| 33496.0|
 6|2021| 1| 445|Food and beverage...| 70026.0| 70921.0|
 7|2021| 1| 4451| Grocery stores| 63171.0| 62919.0|
 8|2021| 1| 446|Health and person...| 29377.0| 29977.0|
 9|2021| 1| 44611|Pharmacies and dr...| 24829.0| 24929.0|
 10|2021| 1| 447| Gasoline stations| 36994.0| 41104.0|
 11|2021| 1| 448|Clothing and clot...| 15385.0| 21093.0|
 12|2021| 1| 4481| Clothing stores| 10432.0| 14135.0|
 13|2021| 1| 44811|Men's clothing st...| 385.0| 494.0|
 14|2021| 1| 44812|Women's clothing ...| 2140.0| 2912.0|
 15|2021| 1| 4482| Shoe stores| 2264.0| 3276.0|
 16|2021| 1| 44831| Jewelry stores| 2415.0| 3359.0|
 17|2021| 1| 451|Sporting goods, h...| 7212.0| 8545.0|
 18|2021| 1| 452|General merchandi...| 58475.0| 65762.0|
 19|2021| 1| 4521| Department stores| 8236.0| 10555.0|
+---+----+-----+---------+--------------------+----------------+--------------+
only showing top 20 rows

In [ ]:
append_NAICS_df = sales_df['NAICScode', 'NAICSname'].distinct()

append_NAICS_df = (append_NAICS_df
    .withColumnRenamed('NAICScode', '2017 NAICS code (NAICS2017)')
    .withColumnRenamed('NAICSname', 'Meaning of NAICS code (NAICS2017_LABEL)'))

append_NAICS_df.show()

+---------------------------+---------------------------------------+
2017 NAICS code (NAICS2017)|Meaning of NAICS code (NAICS2017_LABEL)|
+---------------------------+---------------------------------------+
 446| Health and person...|
 448| Clothing and clot...|
 44611| Pharmacies and dr...|
 441| Motor vehicle and...|
 4481| Clothing stores|
 44812| Women's clothing ...|
 44831| Jewelry stores|
 4529| Other general mer...|
 447| Gasoline stations|
 445| Food and beverage...|
 4441| Building mat. and...|
 452| General merchandi...|
 451| Sporting goods, h...|
 4482| Shoe stores|
 444| Building mat. and...|
 443| Electronics and a...|
 4521| Department stores|
 4451| Grocery stores|
 44811| Men's clothing st...|
 4413| Automotive parts,...|
+---------------------------+---------------------------------------+
only showing top 20 rows

In [ ]:
append_NAICS_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table4) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
sales_df = sales_df.drop('_c0')
# Join Date table and Sales Data so it will have the DateID foreign key (Join on month and year)
sales_and_dateid_df = sales_df.join(dateid_df, ['Year','Month'], 'inner')

#sales_and_dateid_df = sales_and_dateid_df.withColumn('Month',sales_and_dateid_df.Month.cast(IntegerType())) \
    #.withColumn('Year',sales_and_dateid_df..cast(IntegerType()))

cleaned_sales_df = sales_and_dateid_df.drop('Year','Month')
cleaned_sales_df.show()

+---------+--------------------+----------------+--------------+------+
NAICScode| NAICSname|Unadjusted Sales|Adjusted Sales|DateID|
+---------+--------------------+----------------+--------------+------+
 45431| Fuel dealers| 1916.0| 1282.0| 398|
 454| Nonstore retailers| 6860.0| 6515.0| 398|
 45299|All other gen. me...| 2111.0| 2682.0| 398|
 45291|Warehouse clubs a...| 2579.0| 3063.0| 398|
 4529|Other general mer...| 4690.0| 5745.0| 398|
 4521| Department stores| 10306.0| 14134.0| 398|
 452|General merchandi...| 14996.0| 19879.0| 398|
 451|Sporting goods, h...| 3023.0| 3385.0| 398|
 44831| Jewelry stores| 796.0| 1248.0| 398|
 4482| Shoe stores| 1206.0| 1608.0| 398|
 44812|Women's clothing ...| 1873.0| 2631.0| 398|
 44811|Men's clothing st...| 701.0| 840.0| 398|
 4481| Clothing stores| 4852.0| 6758.0| 398|
 448|Clothing and clot...| 6938.0| 9716.0| 398|
 447| Gasoline stations| 12099.0| 12803.0| 398|
 44611|Pharmacies and dr...| 6358.0| 6461.0| 398|
 446|Health and person...| 7258.0| 7421.0| 398|
 4451| Grocery stores| 27306.0| 27835.0| 398|
 445|Food and beverage...| 29589.0| 30437.0| 398|
 4441|Building mat. and...| 7637.0| 9059.0| 398|
+---------+--------------------+----------------+--------------+------+
only showing top 20 rows

In [ ]:
naics_id_df2 = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table4) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
naics_id_df.show()

+-------+---------------------------+---------------------------------------+
NAICSID|2017 NAICS code (NAICS2017)|Meaning of NAICS code (NAICS2017_LABEL)|
+-------+---------------------------+---------------------------------------+
 1| 444130| Hardware stores|
 2| 454110| Electronic shoppi...|
 3| 445210| Meat markets|
 4| 448190| Other clothing st...|
 5| 445220| Fish and seafood ...|
 6| 448310| Jewelry stores|
 7| 448120| Women's clothing ...|
 8| 453910| Pet and pet suppl...|
 9| 441310| Automotive parts ...|
 10| 443141| Household applian...|
 11| 443142| Electronics stores|
 12| 444210| Outdoor power equ...|
 13| 445120| Convenience stores|
 14| 444220| Nursery, garden c...|
 15| 453210| Office supplies a...|
 16| 442299| All other home fu...|
 17| 451130| Sewing, needlewor...|
 18| 445230| Fruit and vegetab...|
 19| 442210| Floor covering st...|
 20| 448130| Children's and in...|
+-------+---------------------------+---------------------------------------+
only showing top 20 rows

In [ ]:
# Join Date table and Sales Data so it will have the DateID foreign key (Join on month and year)
cleaned_sales_df2 = cleaned_sales_df.join(naics_id_df2, cleaned_sales_df['NAICSname'] == naics_id_df2['Meaning of NAICS code (NAICS2017_LABEL)'], 'full')
cleaned_sales_df3 = cleaned_sales_df2.drop('NAICScode', 'NAICSname', '2017 NAICS code (NAICS2017)', 'Meaning of NAICS code (NAICS2017_LABEL)')
cleaned_sales_df3.dtypes

Out[45]: [('Unadjusted Sales', 'string'),
 ('Adjusted Sales', 'string'),
 ('DateID', 'int'),
 ('NAICSID', 'int')]

In [ ]:
# Replace all non int values with nulls
cleaned_sales_df3 = cleaned_sales_df3.withColumn("Unadjusted Sales", \
       when((col("Unadjusted Sales") == "(S)"), None) \
          .otherwise(col("Unadjusted Sales")))

cleaned_sales_df3 = cleaned_sales_df3.withColumn("Adjusted Sales", \
       when((col("Adjusted Sales") == "(S)"), None) \
          .otherwise(col("Adjusted Sales")))
cleaned_sales_df3.dtypes

Out[46]: [('Unadjusted Sales', 'string'),
 ('Adjusted Sales', 'string'),
 ('DateID', 'int'),
 ('NAICSID', 'int')]

In [ ]:
cleaned_sales_df3.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option('truncate', 'true') \
    .mode("overwrite") \
    .option("dbtable", table6) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()